In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
idWork=219598

In [3]:
acce = loadOSCdata.queryOSCapi(OSCid = idWork, output = 'shp', outputFile = 'acce',dataType='accelerometer')
acce.head()

,timestamp,long,lat,pointIndex,geometry,V,tripID
0,2017-05-15 22:28:21.171333,-74.000973,40.695163,136,POINT (-74.00097359999999 40.69516439),1.134069,219598
1,2017-05-15 22:28:22.183394,-74.000974,40.695165,341,POINT (-74.000974575 40.69516552),0.780236,219598
2,2017-05-15 22:28:23.209512,-74.000975,40.695166,546,POINT (-74.00097883000001 40.695164275),0.840056,219598
3,2017-05-15 22:28:24.210884,-74.000982,40.695163,825,POINT (-74.00098446999999 40.69516133),1.034515,219598
4,2017-05-15 22:28:25.182545,-74.000986,40.695160,1216,POINT (-74.000988475 40.695159305),1.447413,219598


In [4]:
bufers = gpd.read_file('../data/nyc-bike-routes/nyc_bike_30mbuffer_epsg=3857.shp')
bikelanes=gpd.read_file('../data/nyc-bike-routes/nyc_bike_routes_2017.shp')

#bikelanes = bikelanes.to_crs(epsg=3857)


#bufers['line'] = bikelanes.geometry


In [5]:
def snapToBikelane(bikelaneDF,bufersDF,pointsDF):
    '''
    this function takes:
    - a point data set (accelerometer or photos)
    - a bikelane buffer geopandas
    - a bikelane geopandas
    and returns a new geopandas with :
    the bikelane ID where the point belongs to  
    the bikeline in geometry
    the new point snap to line in geomtry 
    '''
    #change projection for points, bike shp already in 3857
    pointsDF = pointsDF.to_crs(epsg=3857)
    bufersDF = bufersDF.to_crs(epsg=3857)
    bikelaneDF = bikelaneDF.to_crs(epsg=3857)

    #give the line of the bikelane to the bufersDF
    bufersDF['line'] = bikelaneDF.geometry
    
    #joint points with buffer
    joinData = gpd.sjoin(pointsDF, bufersDF, how="left", op='intersects')
    
    #get unique pointID TIMES MAY CHANGE IN THE NAME WARNING
    allThePoints = pointsDF.pointIndex.unique()
    
    #create empty lists where we store new data
    line = []
    bikelanesID = []
    
    #get point ID duplicated
    duplicates = joinData.pointIndex[joinData.pointIndex.duplicated()].unique()

    
    for i in range(len(allThePoints)):
        #check if the pointIndex is unique:
        if allThePoints[i] not in duplicates:
            #append line from joint to that index
            line.append(joinData.line.loc[joinData.pointIndex == allThePoints[i]].iloc[0])
            bikelanesID.append(joinData.ID_ORIGINA.loc[joinData.pointIndex == allThePoints[i]].iloc[0])

        else:
            #if not, append from the previous id
            line.append(joinData.line.loc[joinData.pointIndex == allThePoints[i-1]].iloc[0])
            bikelanesID.append(joinData.ID_ORIGINA.loc[joinData.pointIndex == allThePoints[i-1]].iloc[0])
    
    pointsDF['line'] = line
    pointsDF['bikelanesID'] = bikelanesID
    
    
    pointOnLine = []

    for i in range(pointsDF.shape[0]):
        try:
            newPoint = pointsDF.line.loc[i].interpolate(pointsDF.line.loc[i].project(pointsDF.geometry.loc[i]))

        except AttributeError:
            newPoint = np.nan

        pointOnLine.append(newPoint)

    pointsDF['pointOnLine'] = pointOnLine   
    
    return pointsDF

In [6]:
pointsDF = snapToBikelane(bikelaneDF = bikelanes, bufersDF = bufers, pointsDF = acce)


In [7]:
pointsDF.head(5)

,timestamp,long,lat,pointIndex,geometry,V,tripID,line,bikelanesID,pointOnLine
0,2017-05-15 22:28:21.171333,-74.000973,40.695163,136,POINT (-8237750.69935848 4967481.926590688),1.134069,219598,NaN,NaN,NaN
1,2017-05-15 22:28:22.183394,-74.000974,40.695165,341,POINT (-8237750.807894984 4967482.092500463),0.780236,219598,NaN,NaN,NaN
2,2017-05-15 22:28:23.209512,-74.000975,40.695166,546,POINT (-8237751.281559418 4967481.909706064),0.840056,219598,NaN,NaN,NaN
3,2017-05-15 22:28:24.210884,-74.000982,40.695163,825,POINT (-8237751.909401344 4967481.47731291),1.034515,219598,NaN,NaN,NaN
4,2017-05-15 22:28:25.182545,-74.000986,40.695160,1216,POINT (-8237752.355235906 4967481.179996749),1.447413,219598,NaN,NaN,NaN


## Export to test


In [8]:
point = pointsDF.loc[:,['timestamp',
 'long',
 'lat',
 'pointIndex',
 'geometry',
 'V']]

line = pointsDF.loc[:,['timestamp',
 'long',
 'lat',
 'pointIndex',
 'V',
 'tripID',
 'line',
 'bikelanesID']]

line.columns = ['timestamp',
 'long',
 'lat',
 'pointIndex',
 'V',
 'tripID',
 'geometry',
 'bikelanesID']

newPoint = pointsDF.loc[:,['timestamp',
 'long',
 'lat',
 'pointIndex',
 'V',
 'tripID',
 'bikelanesID',
 'pointOnLine']]

newPoint.columns = ['timestamp',
 'long',
 'lat',
 'pointIndex',
 'V',
 'tripID',
 'bikelanesID',
 'geometry']

In [9]:
point = point.to_crs(epsg=4326)
point.to_file('point')
line = line.dropna().to_crs(epsg=4326)
line.to_file('line')
newPoint = newPoint.dropna().to_crs(epsg=4326)
newPoint.to_file('newPoint')